# Chapter 6: Teaching Machines to See

## Image Classification

In this, we will train a complex convolution neural network on an image classification dataset.


<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch06-Image-Classification-with-CNNs/6.1.Image_Classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

## Importing necessary libraries

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import requests
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import tensorflow.keras.backend as K
import pickle
from tensorflow.keras.models import load_model, Model

def fix_random_seed(seed):
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

random_seed = 4321
# Fixing the random seed
fix_random_seed(random_seed)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass

2022-07-28 18:20:08.255376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 18:20:08.306682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 18:20:08.307151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Downloading data

For this chapter, we're going to use the tiny-imagenet dataset (200 categories of objects). This is a simplified version of the bigger and harder imagenet dataset (1000 categories of objects).

In [2]:
#Section 6.1 

import os
import requests
import zipfile

# Retrieve the data
if not os.path.exists(os.path.join('data','tiny-imagenet-200.zip')):
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.mkdir('data')
    
    # Write to a file
    with open(os.path.join('data','tiny-imagenet-200.zip'), 'wb') as f:
        f.write(r.content)
else:
    print("The zip file already exists.")
    
if not os.path.exists(os.path.join('data', 'tiny-imagenet-200')):
    with zipfile.ZipFile(os.path.join('data','tiny-imagenet-200.zip'), 'r') as zip_ref:
        zip_ref.extractall('data')
else:
    print("The extracted data already exists")

The zip file already exists.
The extracted data already exists


## Read the label information of the data

The classes in tiny-imagenet are coded by an id (known as `wnid` (WordNetID)). Here we will decode these IDs to get class descriptions of each ID, so we know what we're dealing with

In [3]:
#Section 6.1
#Code listing 6.1

import os
import pandas as pd

# The file paths we're going to use
data_dir = os.path.join('data', 'tiny-imagenet-200')
wnids_path = os.path.join(data_dir, 'wnids.txt')
words_path = os.path.join(data_dir, 'words.txt')

def get_tiny_imagenet_classes(wnids_path, words_path):
    
    # Read the csv files
    # wninds.txt contains the wnids of the data in the dataset
    wnids = pd.read_csv(wnids_path, header=None, squeeze=True)
    # words.txt contains a mapping from wnid to the class description
    words = pd.read_csv(words_path, sep='\t', index_col=0, header=None)
    # Get only the class descriptions corresponding to the wnids in the dataset
    words_200 = words.loc[wnids].rename({1:'class'}, axis=1)
    
    words_200.index.name = 'wnid'
    return words_200.reset_index()

labels = get_tiny_imagenet_classes(wnids_path, words_path)
labels.head(n=25)

/tmp/ipykernel_3572/1569625529.py:16: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  wnids = pd.read_csv(wnids_path, header=None, squeeze=True)


,wnid,class
0,n02124075,Egyptian cat
1,n04067472,reel
2,n04540053,volleyball
3,n04099969,"rocking chair, rocker"
4,n07749582,lemon
5,n01641577,"bullfrog, Rana catesbeiana"
6,n02802426,basketball
7,n09246464,"cliff, drop, drop-off"
8,n07920052,espresso
9,n03970156,"plunger, plumber's helper"


## How many training instances for each class are there?

**Note**: If you run this after separating out the validation data, you'll see 450 instead of 500 in the `n_train` column.

In [4]:
#Section 6.1

import os
def get_image_count(data_dir):  
    """ Counts the number of jpeg files in a given folder"""
    if not os.path.exists(data_dir):
        return 0
    return len([f for f in os.listdir(data_dir) if f.lower().endswith('jpeg')])
    
# Here we use the apply function in conjunction with the get_image_count to get the count of images in each
# subfolder in the train directory
labels["n_train"] = labels["wnid"].apply(lambda x: get_image_count(os.path.join(data_dir, 'train', x, 'images')))
labels.head()

,wnid,class,n_train
0,n02124075,Egyptian cat,500
1,n04067472,reel,500
2,n04540053,volleyball,500
3,n04099969,"rocking chair, rocker",500
4,n07749582,lemon,500


## Look at summary statistics of the `n_train` column

Summary statistics is a great way to get an instant view of a column in a dataframe. It gives us important information like mean/standard deviation, etc.

In [5]:
labels["n_train"].describe()

count    200.0
mean     500.0
std        0.0
min      500.0
25%      500.0
50%      500.0
75%      500.0
max      500.0
Name: n_train, dtype: float64

## Understanding the image dimensions in the dataset

Before moving on to modelling we need to understand the image dimensions (height and width). 

In [6]:
#Section 6.1
#Code listing 6.2

import os
from PIL import Image
import pandas as pd

# A list that will hold image height and width information
image_sizes = []

# We will only look at the first 25 directories (to save time)
for wnid in labels["wnid"].iloc[:25]:
    img_dir = os.path.join('data', 'tiny-imagenet-200', 'train', wnid, 'images')
    for f in os.listdir(img_dir):
        # Only read the file if ends with JPEG
        if f.endswith('JPEG'):
            # Append the height and width to the list
            # e.g. [(img1.width, img1.height), (img2.width, img2.height), ...]
            image_sizes.append(Image.open(os.path.join(img_dir, f)).size)

# Using the format of image_sizes, we can directly create a dataframe
img_df = pd.DataFrame.from_records(image_sizes)
img_df.columns = ["width", "height"]
# Getting summary statistics of all columns
img_df.describe()

,width,height
count,12500.0,12500.0
mean,64.0,64.0
std,0.0,0.0
min,64.0,64.0
25%,64.0,64.0
50%,64.0,64.0
75%,64.0,64.0
max,64.0,64.0


## Bonus: Getting information about pixel values of the images

In [7]:
import os
import numpy as np
from PIL import Image
images = []
for wnid in labels["wnid"].iloc[:2]:
    img_dir = os.path.join('data', 'tiny-imagenet-200', 'train', wnid, 'images')
    for f in os.listdir(img_dir)[:10]:
        if f.endswith('JPEG'):
            img = np.array(Image.open(os.path.join(img_dir, f)))
            images.append((img.min(), img.max(), img.mean()))

img_df = pd.DataFrame.from_records(images)
img_df.columns = ["Min", "Max", "Mean"]
img_df.describe()

,Min,Max,Mean
count,20.000000,20.000000,20.000000
mean,0.650000,-3.150000,115.246729
std,2.300458,8.041177,52.411007
min,0.000000,-37.000000,45.075928
25%,0.000000,-1.000000,83.692464
50%,0.000000,-1.000000,108.461304
75%,0.000000,-1.000000,128.942668
max,10.000000,-1.000000,219.116618


## Creating the data generators

In the dataset, we currently have a,

* Training set: `train` directory
* Testing set: `val` directory

But to properly train a model we need three datasets,

* Training set - Used to train the model
* Validation set - Used to continuously monitor model performance while training
* Testing set - Used to test the model, only after training finishes

Therefore, we will separate a 10% from training data and feed this data through as a separate generator

In [8]:
#Section 6.2

from functools import partial

def get_test_labels_df(test_labels_path):
    """ Reading the test data labels for all files in the test set as a data frame """
    test_df = pd.read_csv(test_labels_path, sep='\t', index_col=None, header=None)
    test_df = test_df.iloc[:,[0,1]].rename({0:"filename", 1:"class"}, axis=1)
    return test_df

def get_train_valid_test_data_generators(batch_size, target_size):
    # Define a Keras ImageDataGenerator with image centering (subtract mean)
    
    image_gen = ImageDataGenerator(samplewise_center=True, validation_split=0.1)

    # Define a training data generator
    partial_flow_func = partial(
        image_gen.flow_from_directory, 
        directory=os.path.join('data','tiny-imagenet-200', 'train'), 
        target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, 
        shuffle=True, seed=random_seed)
    
    # Get the training data subset
    train_gen = partial_flow_func(subset='training')
    # Get the validation data subset
    valid_gen = partial_flow_func(subset='validation')
    
    
    # Define a testing data generator
    # This function uses flow_from_dataframe instead of flow_from_directory
    test_df = get_test_labels_df(os.path.join('data','tiny-imagenet-200',  'val', 'val_annotations.txt'))
    test_gen = image_gen.flow_from_dataframe(
        test_df, directory=os.path.join('data','tiny-imagenet-200',  'val', 'images'), target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=False
    )
    return train_gen, valid_gen, test_gen

def data_gen_aux(gen):    
    # We need to modify our standard data generators to output the same target three times
    for x,y in gen:        
        yield x,(y,y,y)

batch_size = 128
target_size = (56, 56)
# Getting the train,valid, test data generators
train_gen, valid_gen, test_gen = get_train_valid_test_data_generators(batch_size, target_size)
# Modifying the data generators to fit the model targets
train_gen_aux = data_gen_aux(train_gen)
valid_gen_aux = data_gen_aux(valid_gen)
test_gen_aux = data_gen_aux(test_gen)

Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


## Validating the consistency of validation data

In [9]:
from itertools import tee
all_labels = []
n_trials = 10

valid_gen_test = tee(valid_gen, n_trials)

for i in range(n_trials):    
    labels = []
    for j in range(5):
        _, ohe = next(valid_gen_test[i])
        # Convert one hot encoded to class labels
        labels.append(np.argmax(ohe, axis=-1))
        
    # Concat all labels
    labels = np.reshape(np.concatenate(labels, axis=0), (1,-1))        
    all_labels.append(labels)

# Concat all labels accross all trials
all_labels = np.concatenate(all_labels, axis=0)

# Assert the labels are equal across all trials
assert np.all(np.all(all_labels == all_labels[0,:], axis=0)), "Labels across multiple trials were not equal"
print("Successful! Labels across all trials were consistent.")

Successful! Labels across all trials were consistent.


## Defining the Inception net v1

Here we will be creating the Inception net v1 model using Keras Functional API

### Funtions that encapsulate various components of the Inception net v1 model

In [10]:
# Section 6.3
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K

# Code listing 6.3
def stem(inp):
    conv1 = Conv2D(64, (7,7), strides=(1,1), activation='relu', padding='same')(inp)
    maxpool2 = MaxPool2D((3,3), strides=(2,2), padding='same')(conv1)
    lrn3 = Lambda(lambda x: tf.nn.local_response_normalization(x))(maxpool2)

    conv4 = Conv2D(64, (1,1), strides=(1,1), padding='same')(lrn3)
    conv5 = Conv2D(192, (3,3), strides=(1,1), activation='relu', padding='same')(conv4)
    lrn6 = Lambda(lambda x: tf.nn.local_response_normalization(x))(conv5)

    maxpool7 = MaxPool2D((3,3), strides=(1,1), padding='same')(lrn6)

    return maxpool7

# Code listing 6.4
def inception(inp, n_filters):

    # 1x1 layer
    # init argument defaults to glorot_uniform
    out1 = Conv2D(n_filters[0][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)

    # 1x1 followed by 3x3
    out2_1 = Conv2D(n_filters[1][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)
    out2_2 = Conv2D(n_filters[1][1], (3,3), strides=(1,1), activation='relu', padding='same')(out2_1)

    # 1x1 followed by 5x5
    out3_1 = Conv2D(n_filters[2][0], (1,1), strides=(1,1), activation='relu', padding='same')(inp)
    out3_2 = Conv2D(n_filters[2][1], (5,5), strides=(1,1), activation='relu', padding='same')(out3_1)

    # 3x3 (pool) followed by 1x1
    out4_1 = MaxPool2D((3,3), strides=(1,1), padding='same')(inp)
    out4_2 = Conv2D(n_filters[3][0], (1,1), strides=(1,1), activation='relu', padding='same')(out4_1)

    out = Concatenate(axis=-1)([out1, out2_2, out3_2, out4_2])
    return out

# Code listing 6.5
def aux_out(inp,name=None):    
    avgpool1 = AvgPool2D((5,5), strides=(3,3), padding='valid')(inp)
    conv1 = Conv2D(128, (1,1), activation='relu', padding='same')(avgpool1)
    flat = Flatten()(conv1)
    dense1 = Dense(1024, activation='relu')(flat)    
    aux_out = Dense(200, activation='softmax', name=name)(dense1)
    return aux_out


### Defining the final model

In [11]:
# Section 6.3
# Code listing 6.6
def inception_v1():
    
    K.clear_session()
    
    inp = Input(shape=(56,56,3))
    stem_out = stem(inp)
    inc_3a = inception(stem_out, [(64,),(96,128),(16,32),(32,)])
    inc_3b = inception(inc_3a, [(128,),(128,192),(32,96),(64,)])

    maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')(inc_3b)

    inc_4a = inception(maxpool, [(192,),(96,208),(16,48),(64,)])
    inc_4b = inception(inc_4a, [(160,),(112,224),(24,64),(64,)])

    aux_out1 = aux_out(inc_4a, name='aux1')

    inc_4c = inception(inc_4b, [(128,),(128,256),(24,64),(64,)])
    inc_4d = inception(inc_4c, [(112,),(144,288),(32,64),(64,)])
    inc_4e = inception(inc_4d, [(256,),(160,320),(32,128),(128,)])
    
    maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')(inc_4e)
    
    aux_out2 = aux_out(inc_4d, name='aux2')

    inc_5a = inception(maxpool, [(256,),(160,320),(32,128),(128,)])
    inc_5b = inception(inc_5a, [(384,),(192,384),(48,128),(128,)])
    avgpool1 = AvgPool2D((7,7), strides=(1,1), padding='valid')(inc_5b)

    flat_out = Flatten()(avgpool1)
    out_main = Dense(200, activation='softmax', name='final')(flat_out)

    model = Model(inputs=inp, outputs=[out_main, aux_out1, aux_out2])
    model.compile(loss='categorical_crossentropy', 
                       optimizer='adam', metrics=['accuracy'])
    return model

model = inception_v1()
model.summary()

2022-07-28 18:21:21.391556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-28 18:21:21.392599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 18:21:21.393331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 18:21:21.393885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 56, 56, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 56, 56, 64)   9472        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 28, 28, 64)   0           ['conv2d[0][0]']                 
                                                                                                  
 lambda (Lambda)                (None, 28, 28, 64)   0           ['max_pooling2d[0][0]']          
                                                                                              

                                                                                                  
 conv2d_21 (Conv2D)             (None, 14, 14, 160)  82080       ['concatenate_2[0][0]']          
                                                                                                  
 conv2d_23 (Conv2D)             (None, 14, 14, 224)  226016      ['conv2d_22[0][0]']              
                                                                                                  
 conv2d_25 (Conv2D)             (None, 14, 14, 64)   38464       ['conv2d_24[0][0]']              
                                                                                                  
 conv2d_26 (Conv2D)             (None, 14, 14, 64)   32832       ['max_pooling2d_6[0][0]']        
                                                                                                  
 concatenate_3 (Concatenate)    (None, 14, 14, 512)  0           ['conv2d_21[0][0]',              
          

 conv2d_49 (Conv2D)             (None, 7, 7, 320)    461120      ['conv2d_48[0][0]']              
                                                                                                  
 conv2d_51 (Conv2D)             (None, 7, 7, 128)    102528      ['conv2d_50[0][0]']              
                                                                                                  
 conv2d_52 (Conv2D)             (None, 7, 7, 128)    106624      ['max_pooling2d_11[0][0]']       
                                                                                                  
 concatenate_7 (Concatenate)    (None, 7, 7, 832)    0           ['conv2d_47[0][0]',              
                                                                  'conv2d_49[0][0]',              
                                                                  'conv2d_51[0][0]',              
                                                                  'conv2d_52[0][0]']              
          

## Training the model

In [12]:
# Section 6.4

def get_steps_per_epoch(n_data, batch_size):
    """ Given the data size and batch size, gives the number of steps to travers the full dataset """
    if n_data%batch_size==0:
        return int(n_data/batch_size)
    else:
        return int(n_data*1.0/batch_size)+1

In [13]:
# Section 6.4
# Code listing 6.7

from tensorflow.keras.callbacks import CSVLogger
import time
import os

# Create a directory called eval which stores model performance
if not os.path.exists('eval'):
    os.mkdir('eval')
    
# This will automatically log model performan to this file
csv_logger = CSVLogger(os.path.join('eval','1_eval_base.log'))
    
t1 = time.time() # Starting time

# Train the model, not how we are specifying steps_per_epoch and validation_steps
# to prevent the model from training forever
# We are also using the data generators (not actual data loaded to memory) to train the model
history = model.fit(
    train_gen_aux, validation_data=valid_gen_aux, 
    steps_per_epoch=get_steps_per_epoch(0.9*500*200,batch_size), 
    validation_steps=get_steps_per_epoch(0.1*500*200,batch_size),
    epochs=50, callbacks=[csv_logger]
)
t2 = time.time() # Ending time

# Print time it took
print("It took {} seconds to complete the training".format(t2-t1))

Epoch 1/50


2022-07-28 18:21:29.941693: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2022-07-28 18:21:31.479960: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


704/704 [==============================] - 193s 262ms/step - loss: 14.5547 - final_loss: 4.9425 - aux1_loss: 4.7561 - aux2_loss: 4.8562 - final_accuracy: 0.0258 - aux1_accuracy: 0.0474 - aux2_accuracy: 0.0368 - val_loss: 13.2079 - val_final_loss: 4.5422 - val_aux1_loss: 4.2577 - val_aux2_loss: 4.4080 - val_final_accuracy: 0.0607 - val_aux1_accuracy: 0.0979 - val_aux2_accuracy: 0.0763
Epoch 2/50
704/704 [==============================] - 184s 262ms/step - loss: 12.2247 - final_loss: 4.2296 - aux1_loss: 3.9186 - aux2_loss: 4.0766 - final_accuracy: 0.0948 - aux1_accuracy: 0.1462 - aux2_accuracy: 0.1189 - val_loss: 11.5159 - val_final_loss: 3.9503 - val_aux1_loss: 3.7262 - val_aux2_loss: 3.8394 - val_final_accuracy: 0.1310 - val_aux1_accuracy: 0.1758 - val_aux2_accuracy: 0.1487
Epoch 3/50
704/704 [==============================] - 186s 265ms/step - loss: 10.8330 - final_loss: 3.7453 - aux1_loss: 3.4681 - aux2_loss: 3.6196 - final_accuracy: 0.1679 - aux1_accuracy: 0.2160 - aux2_accuracy: 0.

Epoch 22/50
704/704 [==============================] - 187s 265ms/step - loss: 1.1569 - final_loss: 0.3784 - aux1_loss: 0.2911 - aux2_loss: 0.4873 - final_accuracy: 0.8798 - aux1_accuracy: 0.9046 - aux2_accuracy: 0.8446 - val_loss: 21.4164 - val_final_loss: 6.4535 - val_aux1_loss: 7.9431 - val_aux2_loss: 7.0198 - val_final_accuracy: 0.2738 - val_aux1_accuracy: 0.2488 - val_aux2_accuracy: 0.2472
Epoch 23/50
704/704 [==============================] - 187s 265ms/step - loss: 1.0581 - final_loss: 0.3450 - aux1_loss: 0.2679 - aux2_loss: 0.4452 - final_accuracy: 0.8900 - aux1_accuracy: 0.9139 - aux2_accuracy: 0.8570 - val_loss: 21.2719 - val_final_loss: 6.2677 - val_aux1_loss: 8.0720 - val_aux2_loss: 6.9322 - val_final_accuracy: 0.2742 - val_aux1_accuracy: 0.2329 - val_aux2_accuracy: 0.2406
Epoch 24/50
704/704 [==============================] - 187s 265ms/step - loss: 1.0091 - final_loss: 0.3373 - aux1_loss: 0.2442 - aux2_loss: 0.4276 - final_accuracy: 0.8929 - aux1_accuracy: 0.9197 - aux2_a

Epoch 43/50
704/704 [==============================] - 189s 268ms/step - loss: 0.6111 - final_loss: 0.2269 - aux1_loss: 0.1400 - aux2_loss: 0.2441 - final_accuracy: 0.9293 - aux1_accuracy: 0.9551 - aux2_accuracy: 0.9218 - val_loss: 26.4504 - val_final_loss: 7.6069 - val_aux1_loss: 10.0566 - val_aux2_loss: 8.7869 - val_final_accuracy: 0.2740 - val_aux1_accuracy: 0.2384 - val_aux2_accuracy: 0.2408
Epoch 44/50
704/704 [==============================] - 188s 266ms/step - loss: 0.6440 - final_loss: 0.2394 - aux1_loss: 0.1496 - aux2_loss: 0.2549 - final_accuracy: 0.9251 - aux1_accuracy: 0.9523 - aux2_accuracy: 0.9180 - val_loss: 26.9409 - val_final_loss: 8.0343 - val_aux1_loss: 10.0430 - val_aux2_loss: 8.8636 - val_final_accuracy: 0.2732 - val_aux1_accuracy: 0.2430 - val_aux2_accuracy: 0.2514
Epoch 45/50
704/704 [==============================] - 187s 265ms/step - loss: 0.6006 - final_loss: 0.2149 - aux1_loss: 0.1460 - aux2_loss: 0.2397 - final_accuracy: 0.9339 - aux1_accuracy: 0.9528 - aux2

## Bonus: Metrics of the model

This is a multi output model. So it pays off to check what are the metrics that the model uses. You can get metric names by calling `model.metrics_names` as follows.

In [14]:
print(model.metrics_names)

['loss', 'final_loss', 'aux1_loss', 'aux2_loss', 'final_accuracy', 'aux1_accuracy', 'aux2_accuracy']


## Saving the trained model

In [15]:
# Save the model to models directory
if not os.path.exists('models'):
    os.mkdir("models")
model.save(os.path.join('models', 'inception_v1_base.h5'))

## Get test accuracy of the trained model

In [16]:
# Section 6.4

# Load the model from disk
model = load_model(os.path.join('models','inception_v1_base.h5'))

# Evaluate the model
test_res = model.evaluate(test_gen_aux, steps=get_steps_per_epoch(500*50, batch_size))

# Print the results as a dictionary {<metric name>: <value>}
test_res_dict = dict(zip(model.metrics_names, test_res))
print(test_res_dict)

196/196 [==============================] - 17s 85ms/step - loss: 27.5954 - final_loss: 8.3132 - aux1_loss: 10.1646 - aux2_loss: 9.1176 - final_accuracy: 0.2670 - aux1_accuracy: 0.2454 - aux2_accuracy: 0.2419
{'loss': 27.595443725585938, 'final_loss': 8.31319522857666, 'aux1_loss': 10.164627075195312, 'aux2_loss': 9.11762809753418, 'final_accuracy': 0.2669723331928253, 'aux1_accuracy': 0.24537484347820282, 'aux2_accuracy': 0.2418757975101471}
